In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras

model = keras.models.load_model('Model_v1.keras')

In [22]:
from datetime import datetime, timedelta

now = datetime.now()
time_delta = timedelta(days=200)
start = now - time_delta
end = now

print(f"From: {start} to {end}")

From: 2025-03-19 11:15:59.834019 to 2025-10-05 11:15:59.834019


In [23]:
ticker = "^NSEI"
df = yf.download(ticker, start, end)
# Calculate RSI (Relative Strength Index)
def calculate_rsi(data, window=14):
    diff = data.diff(1).dropna()
    gain = diff.mask(diff < 0, 0)
    loss = diff.mask(diff > 0, 0).abs()
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['RSI'] = calculate_rsi(df['Close'])

# Calculate MACD (Moving Average Convergence Divergence)
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    short_ema = data.ewm(span=short_window, adjust=False).mean()
    long_ema = data.ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal_line = macd.ewm(span=signal_window, adjust=False).mean()
    return macd, signal_line

macd, signal_line = calculate_macd(df['Close'])
df['MACD'] = macd
df['MACD_Signal'] = signal_line

# Calculate EMA 50
df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()

# Calculate SMA 20
df['SMA_20'] = df['Close'].rolling(window=20).mean()

# Calculate Stochastic RSI
def calculate_stoch_rsi(data, window=14, k_window=3, d_window=3):
    rsi = calculate_rsi(data, window)
    min_rsi = rsi.rolling(window=window).min()
    max_rsi = rsi.rolling(window=window).max()
    stoch_rsi = (rsi - min_rsi) / (max_rsi - min_rsi)
    k_line = stoch_rsi.rolling(window=k_window).mean()
    d_line = k_line.rolling(window=d_window).mean()
    return k_line, d_line

df['StochRSI_K'], df['StochRSI_D'] = calculate_stoch_rsi(df['Close'])

df


C:\Users\Krish Patel\AppData\Local\Temp\ipykernel_9192\2771165655.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start, end)
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume,RSI,MACD,MACD_Signal,EMA_50,SMA_20,StochRSI_K,StochRSI_D
Ticker,^NSEI,^NSEI,^NSEI,^NSEI,^NSEI,,,,,,,
Date,,,,,,,,,,,,
2025-03-19,22907.599609,22940.699219,22807.949219,22874.949219,324000,NaN,0.000000,0.000000,22907.599609,NaN,NaN,NaN
2025-03-20,23190.650391,23216.699219,22973.949219,23036.599609,313700,NaN,22.579550,4.515910,22918.699640,NaN,NaN,NaN
2025-03-21,23350.400391,23402.699219,23132.800781,23168.250000,541000,NaN,52.756353,14.163998,22935.629081,NaN,NaN,NaN
2025-03-24,23658.349609,23708.750000,23433.500000,23515.400391,311900,NaN,100.363685,31.403936,22963.971063,NaN,NaN,NaN
2025-03-25,23668.650391,23869.599609,23601.400391,23751.500000,338200,NaN,137.340846,52.591318,22991.605546,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-26,24654.699219,24868.599609,24629.449219,24818.550781,291500,46.075050,45.630939,83.106478,24917.011977,24974.642480,0.197778,0.371172
2025-09-29,24634.900391,24791.300781,24606.199219,24728.550781,394400,41.847517,11.730437,68.831270,24905.948778,24975.134961,0.068742,0.215889


In [24]:
df.dropna(inplace=True)
df = df.reset_index(drop=True)
df.shape

(104, 12)

In [ ]:
# Select only the 9 features the model was trained on
feature_cols = [
	('Open', '^NSEI'),
	('Close', '^NSEI'),
    ('RSI', ''),
	('MACD', ''),
	('MACD_Signal', ''),
	('EMA_50', ''),
    ('SMA_20', ''),
	('StochRSI_K', ''),
	('StochRSI_D', ''),
]

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[feature_cols])

# Prepare the input for prediction
x_input = np.array([scaled_data[-100:]])  # Last 100 rows
x_input = x_input.reshape(1, 100, 9)  # Reshape to (1, 100, 9)

# Predict today's price
y_predicted_scaled = model.predict(x_input)

dummy_array = np.zeros(shape=(len(y_predicted_scaled), 9))

dummy_array[:, 0] = y_predicted_scaled[:, 0] # Predicted 'Close'
dummy_array[:, 1] = y_predicted_scaled[:, 1] # Predicted 'Open'

predicted_prices = scaler.inverse_transform(dummy_array)

# --- Display the results ---
print(f"Predicted Close Price for Today: {predicted_prices[0, 0]:.2f}")
print(f"Predicted Open Price for Today: {predicted_prices[0, 1]:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
Predicted Close Price for Today: 24904.76
Predicted Open Price for Today: 24895.31
